In [ ]:
# !pip install tensorflow_text
# !pip install tensorflow_hub

In [ ]:
import pandas as pd


In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder

df = pd.read_csv('major_data_cleaned_300.csv')
df['combined_features'] = df['Description'] + ' ' + df['Mata Kuliah']

# Tokenization
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(df['combined_features'])

# Converting text to sequences
sequences = tokenizer.texts_to_sequences(df['combined_features'])
padded_sequences = pad_sequences(sequences, maxlen=100)

# Target labels
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(df['NAMA'])


In [ ]:
from sklearn.model_selection import train_test_split
import numpy as np

# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(padded_sequences, y_encoded, test_size=0.2, random_state=42)
y_train = np.array(y_train)
y_val = np.array(y_val)

In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, Dense, GlobalAveragePooling1D, LayerNormalization, MultiHeadAttention, Dropout

# Transformer Encoder Layer
class TransformerBlock(tf.keras.layers.Layer):
    def __init__(self, embed_dim, num_heads, ff_dim, rate=0.1):
        super(TransformerBlock, self).__init__()
        self.att = MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)
        self.ffn = tf.keras.Sequential(
            [Dense(ff_dim, activation="relu"), Dense(embed_dim)]
        )
        self.layernorm1 = LayerNormalization(epsilon=1e-6)
        self.layernorm2 = LayerNormalization(epsilon=1e-6)
        self.dropout1 = Dropout(rate)
        self.dropout2 = Dropout(rate)

    def call(self, inputs, training):
        attn_output = self.att(inputs, inputs)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(inputs + attn_output)
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        return self.layernorm2(out1 + ffn_output)

# Building the Model
def create_model(vocab_size, embed_dim, num_heads, ff_dim, num_classes):
    inputs = Input(shape=(100,))
    embedding_layer = Embedding(vocab_size, embed_dim)
    x = embedding_layer(inputs)
    transformer_block = TransformerBlock(embed_dim, num_heads, ff_dim)
    x = transformer_block(x)
    x = GlobalAveragePooling1D()(x)
    x = Dropout(0.1)(x)
    outputs = Dense(num_classes, activation='softmax')(x)

    model = Model(inputs=inputs, outputs=outputs)
    return model

# Model parameters
vocab_size = 5000
embed_dim = 128
num_heads = 4
ff_dim = 128
num_classes = len(df['NAMA'].unique())

model = create_model(vocab_size, embed_dim, num_heads, ff_dim, num_classes)
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.summary()


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 100)]             0         
                                                                 
 embedding (Embedding)       (None, 100, 128)          640000    
                                                                 
 transformer_block (Transfo  (None, 100, 128)          297344    
 rmerBlock)                                                      
                                                                 
 global_average_pooling1d (  (None, 128)               0         
 GlobalAveragePooling1D)                                         
                                                                 
 dropout_2 (Dropout)         (None, 128)               0         
                                                                 
 dense_2 (Dense)             (None, 294)               37926 

In [ ]:
y_val

array([246, 165,   5,  51, 217, 119,  87, 190,  98, 221,  50, 131, 253,
       132, 255, 276, 104, 162,  63,  59,  79, 212, 145, 252, 278,  45,
        82,   2, 161,  30, 260, 271, 108,  76, 263, 188, 216, 207, 280,
       209, 121, 229,   8,  93, 289,  75, 191, 186,  26,  88, 248, 273,
       167, 151, 222,  29,   9, 155, 227])

In [ ]:
history = model.fit(
    X_train, y_train,
    epochs=200,
    validation_data=(X_val, y_val),
    batch_size=16
)

# Save label tokenizer for inference
# import pickle
# with open('label_tokenizer.pkl', 'wb') as f:
#     pickle.dump(label_seq, f)

Epoch 1/200
15/15 [==============================] - 7s 136ms/step - loss: 5.9204 - accuracy: 0.0000e+00 - val_loss: 6.1231 - val_accuracy: 0.0000e+00
Epoch 2/200
15/15 [==============================] - 2s 134ms/step - loss: 5.3926 - accuracy: 0.0426 - val_loss: 6.2091 - val_accuracy: 0.0000e+00
Epoch 3/200
15/15 [==============================] - 1s 66ms/step - loss: 5.0121 - accuracy: 0.3064 - val_loss: 6.4573 - val_accuracy: 0.0000e+00
Epoch 4/200
15/15 [==============================] - 1s 66ms/step - loss: 4.5825 - accuracy: 0.5447 - val_loss: 6.6633 - val_accuracy: 0.0000e+00
Epoch 5/200
15/15 [==============================] - 1s 80ms/step - loss: 4.0445 - accuracy: 0.6894 - val_loss: 6.8093 - val_accuracy: 0.0000e+00
Epoch 6/200
15/15 [==============================] - 1s 60ms/step - loss: 3.3656 - accuracy: 0.7319 - val_loss: 6.7752 - val_accuracy: 0.0000e+00
Epoch 7/200
15/15 [==============================] - 1s 46ms/step - loss: 2.6477 - accuracy: 0.7702 - val_loss: 7.0197

In [ ]:
def recommend_courses(input_text, model, tokenizer, label_encoder, top_n=10):
    seq = tokenizer.texts_to_sequences([input_text])
    padded_seq = pad_sequences(seq, maxlen=100)
    predictions = model.predict(padded_seq)
    top_indices = predictions[0].argsort()[-top_n:][::-1]
    recommendations = label_encoder.inverse_transform(top_indices)
    return recommendations.tolist()

input_text = "ahli kimia"
top_recommendations = recommend_courses(input_text, model, tokenizer, label_encoder)
print("Top-10 recommended courses:", top_recommendations)


1/1 [==============================] - 0s 19ms/step
Top-10 recommended courses: ['KEWIRAUSAHAAN', 'PEND. MATEMATIKA', 'TEKNIK INFORMATIKA', 'INFORMATIKA', 'PENDIDIKAN GURU SD', 'TEKNOLOGI INDUSTRI PERTANIAN', 'SENI KULINER DAN PENGOLAHAN JASA MAKANAN', 'TATA BOGA', 'ILMU PENDIDIKAN AGAMA ISLAM', 'SASTRA JEPANG']
